# Operations Research Tutorial using PulP

## Integer Programming

### Problem 1

In [ ]:
from pulp import *

In [ ]:
identifiers = ['A', 'B', 'C', 'D', 'E']
prices = dict(zip(identifiers, [100.0, 99.0, 100.5, 101.5, 200.0]))

In [ ]:
prob = LpProblem(name='Minimalist example', sense=LpMaximize)

x = LpVariable.dicts(name='x', indexs=identifiers, lowBound=0, 
                     upBound=1, cat='Integer')

prob += lpSum([x[i]*prices[i] for i in identifiers])

prob += lpSum([x[i] for i in identifiers]) == 2

In [ ]:
prob.solve(solver=GUROBI())

for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Status:', LpStatus[prob.status])
print(value(prob.objective))

### Problem 3

In [ ]:
from pulp import *

In [ ]:
prob = LpProblem(name='Profit maximising problem', sense=LpMaximize)

In [ ]:
x1 = LpVariable(name='x1', lowBound=0, cat='Integer')
x2 = LpVariable(name='x2', lowBound=0, cat='Integer')

prob += 30000*x1 + 45000*x2, 'Profit'

prob += 3*x1 + 4*x2 <= 30
prob += 5*x1 + 6*x2 <= 60
prob += 1.5*x1 + 3*x2 <= 21

In [ ]:
prob.solve(solver=GUROBI())

print(LpStatus[prob.status])

# Print our decision variable values
print('Production of Car x1 = {}'.format(x1.varValue))
print('Production of Car x2 = {}'.format(x2.varValue))

# Print our objective function value
print(value(prob.objective))

### Knapsack Problem

In [ ]:
from pulp import *

In [ ]:
model = LpProblem('knapsack problem', LpMaximize)

In [ ]:
x1 = LpVariable('x1', lowBound=0, upBound=1, cat=LpInteger)
x2 = LpVariable('x2', lowBound=0, upBound=1, cat=LpInteger)
x3 = LpVariable('x3', lowBound=0, upBound=1, cat=LpInteger)

In [ ]:
model += 7*x1 + 8*x2 + 3*x3
model += 3*x1 + 4*x2 + 2*x3 <= 6

In [ ]:
model.solve(GUROBI())

print('Status', LpStatus[model.status])
print(value(model.objective))
for v in model.variables():
    print(v.name, '=', v.varValue)

### Knapsack Problem II

In [ ]:
from pulp import *

In [ ]:
items = ['A', 'B', 'C']
weights = [3, 4, 2]
values = [7, 8, 3]

In [ ]:
prob = LpProblem('knapsack problem', LpMaximize)

# Define decision variables
decision_variables = []
for num, i in enumerate(items):
    var_str = str('x' + str(num))
    variables = LpVariable(str(var_str), lowBound=0, upBound=1, cat='Integer')
    decision_variables.append(variables)
print(decision_variables)

# Define objective function
objective_function = ''
for u, uf in enumerate(values):
    for d, dv in enumerate(decision_variables):
        if u == d:
            objective_function += uf * dv
        
prob += objective_function
print(objective_function)

# Define constraint
capacity = 6
constraint = ''
for w, wt in enumerate(weights):
    for d, dv in enumerate(decision_variables):
        if w == d:
            constraint += wt * dv
            
prob += constraint <= capacity
print(constraint)

In [ ]:
prob.solve(GUROBI())

print('Status', LpStatus[prob.status])
print(value(prob.objective))
for v in prob.variables():
    print(v.name, '=', v.varValue)

In [ ]:
from pulp import *

In [ ]:
data = {
    ("SMITH"): [6,8,30,6,20], ("JOHNSON"): [6,8,50,0,24], 
    ('WILLIAMS'): [6,8,30,0,24], ('JONES'): [6,8,30,0,24], 
    ('BROWN'): [6,8,40,0,24], ('DAVIS'): [6,8,50,0,24],
    ('MILLER'): [6,8,45,6,18], ('WILSON'): [6,8,30,0,24], 
    ('MOORE'): [6,8,35,0,24], ('TAYLOR'): [6,8,40,0,24], 
    ('ANDERSON'): [2,3,60,0,6], ('THOMAS'): [2,4,40,0,24],
    ('JACKSON') :[2,4,60,8,16], ('WHITE'): [2,6,55,0,24], 
    ('HARRIS'): [2,6,45,0,24], ('MARTIN'): [2,3,40,0,24], 
    ('THOMPSON'): [2,5,50,12,24], ('GARCIA'): [2,4,50,0,24],
    ('MARTINEZ'): [2,4,40,0,24], ('ROBINSON'): [2,5,50,0,24]
}

required = [1,1,2,3,6,6,7,8,9,8,8,8,7,6,6,5,5,4,4,3,2,2,2,2]

time = 24

In [ ]:
employee = list(data.keys())
mins, maxs, costs, avs, ave = splitDict(data)

In [ ]:
x = {}
for d in employee:
    for i in range(time):
        for j in range(i+1, time+1):
            x[d,i,j] = LpVariable(name='x_%s%d%d'%(d,i,j), cat='Binary')
            
staff_number = {}
for t in range(time):
    staff_number[t] = LpVariable(name='staffNumber_%d'%t, cat='Integer', lowBound=required[t])

In [ ]:
prob = LpProblem('Work Schedule', LpMinimize)

In [ ]:
prob += lpSum(lpSum(lpSum((j-i) * x[d,i,j] * costs[d] for j in range(i+1,time+1)) \
                    for i in range(time)) for d in employee)

In [ ]:
for d in employee:
    prob += (lpSum(lpSum(x[d,i,j] for j in range(i+1,ave[d]+1)if min[d] <= \
                         (j-i) <= max[d]) for i in range(avs[d],ave[d])) <= 1)
    
    prob += (lpSum(lpSum(x[d,i,j] for j in range(i+1,time+1)) for i in range(time)) <= \
             lpSum(quicksum(x[d,i,j] for j in range(i+1,ave[d]+1) if min[d] <= (j-i) <= \
                            max[d]) for i in range(avs[d],ave[d])))

    
    
for d in employee:
    m.addConstr(quicksum(quicksum(x[d,i,j] for j in range(i+1,ave[d]+1)if min[d] <= (j-i) <= max[d])for i in range(avs[d],ave[d]))<=1)
    m.addConstr(quicksum(quicksum(x[d,i,j] for j in range(i+1,t+1))for i in range(t))<=quicksum(quicksum(x[d,i,j] for j in range(i+1,ave[d]+1)
    if min[d] <= (j-i) <= max[d])for i in range(avs[d],ave[d])))
    
    
for c in range(t):
    m.addConstr(quicksum(quicksum(quicksum(x[d,i,j] for j in range(i+1,t+1)if i <= c <j) for i in range(t))for d in employee)==staffNumber[c])
D101=quicksum(quicksum(x['ANDERSON',i,j] for j in range(i+1,7)if min["ANDERSON"]<=(j-i)<=max["ANDERSON"])for i in range(0,7))
D102=quicksum(quicksum(x['ANDERSON',i,j] for j in range(i+1,21))for i in range(18,21))
m.addConstr(D101+D102<=1,"F")
m.addConstr(quicksum(quicksum(x['ANDERSON',i,j] for j in range(i+1,t+1))for i in range(t))<=D101+D102)